## 要件
- [x] pandasで必要なデータフレームを取得
- [x] データフレームを整形する
- [x] ユーザーに1-5の数字を入力してもらう
- [x] 数字に応じて柔軟にデータを取得
- [x] 結果を可視化する
- [x] ユーザーにファイル名を入力してもらう
- [x] CSVに保存
- [x] バリデーションチェック

In [24]:
# 標準ライブラリ
from pathlib import Path
from typing import Union

# 外部ライブラリ
import pandas as pd
import plotly.express as px

In [25]:
# 定数定義
MAX_DATA_FRAME_COUNT = 5
DATA_FRAME_START_ROW = 3
SELECTED_COLUMNS = [0, 3, 6, 8, 12, 24]

In [26]:
class UserInterruptException(Exception):
    """ユーザーによって処理が中断されたことを示す例外"""
    pass

In [ ]:
def get_number() -> Union[int, None]:
    """ユーザーにデータフレームを選択する数値を入力してもらう処理"""
    
    check_num_list = [1, 2, 3, 4, 5]

    while True:
        try:
            user_input = input("1 ~ 5の数字を入力してください")
            
            # 空文字かどうかをチェックする
            if not user_input.strip():
                print("何も入力されませんでした。処理を中断します。")
                return None
            
            # 空文字ではないときのみ整数変換処理をする
            input_num = int(user_input) # 数字以外だとValueError
            
            # 1~5の範囲内か
            if input_num not in check_num_list:
                print("1 ~ 5以外の値が入力されました。入力し直してください。")
                continue
            
            return input_num
            
        except ValueError:
            # 想定は数字以外の入力がされた場合
            # 注意: 空文字は上でチェック済みなのでここを通らない
            print("数字を入力して下さい。")
            continue # 再ループ
        
        except Exception as e:
            print(f"予期せぬエラーが発生しました: {type(e).__name__}")
            return None


def get_file_name() -> Union[str, None]:
    """ユーザーに保存するファイル名を入力してもらう処理"""
    
    while True:
        try:
            user_input = input("保存するファイル名を入力して下さい")
            
            # スペースのみの入力も空文字として扱い空文字チェックする
            if not user_input.strip():
                print("何も入力されませんでした。処理を中断します。")
                return None
            
            return user_input
        
        except Exception as e:
            print(f"予期せぬエラーが発生しました: {type(e).__name__}")
            return None

In [28]:
# pandasで表を取得する
url = "https://www.data.jma.go.jp/stats/data/mdrr/synopday/data1s.html"

# html内の全てのデータフレームが取得される
get_dfs = pd.read_html(url)
# lenすればいくつデータフレームがあるか確認できる
#print(len(get_dfs)) # 9

# 今回使いたいデータフレームを取得
target_dfs = get_dfs[0:MAX_DATA_FRAME_COUNT]

In [29]:
# ユーザーに数値入力してもらう
input_num = get_number()

# 値がNoneの場合は処理を中断する
if input_num is None:
    raise UserInterruptException("ユーザーによって処理が中断されました。")

# ユーザーにファイル名入力してもらう
input_file_name = get_file_name()

# 値がNoneの場合は処理を中断する
if input_file_name is None:
    raise UserInterruptException("ユーザーによって処理が中断されました。")

# 取得するデータフレームを限定させる
target_df = target_dfs[input_num - 1].iloc[DATA_FRAME_START_ROW:, SELECTED_COLUMNS]
# ヘッダーの調整
header_name = ["拠点", "気圧(hPa)", "最高気温(℃)", "最低気温(℃)", "湿度(%)", "降水量(mm)"]
target_df.columns = header_name

# 値のゴミを取り除いて数値に変換する処理
num_cols = ["気圧(hPa)", "最高気温(℃)", "最低気温(℃)", "湿度(%)", "降水量(mm)"]

for col in num_cols:
    target_df[col] = target_df[col].astype(str).str.replace("]", "", regex=False)
    target_df[col] = pd.to_numeric(target_df[col], errors="coerce")

# 一応拠点列も明示的に文字列型に設定
target_df["拠点"] = target_df["拠点"].astype(str)
# インデックスを0から始まる連番に振りなおす
target_df = target_df.reset_index(drop=True)

target_df

,拠点,気圧(hPa),最高気温(℃),最低気温(℃),湿度(%),降水量(mm)
0,静岡,1018.4,18.6,6.9,30,NaN
1,浜松,1019.8,17.1,5.6,35,NaN
2,御前崎,1019.3,16.3,9.3,40,NaN
3,三島,1019.0,17.0,5.7,42,NaN
4,石廊崎,1019.2,16.9,9.5,45,NaN
5,網代,1019.7,13.6,8.6,54,0.0
6,富士山,NaN,-11.1,-15.4,10,NaN
7,名古屋,1020.8,15.3,3.5,36,NaN
8,伊良湖,1020.4,15.5,6.2,38,NaN
9,岐阜,1020.7,15.5,2.5,36,NaN


In [30]:
# 棒グラフの作成
fig = px.bar(
    target_df,
    x=["最高気温(℃)", "最低気温(℃)"],
    y="拠点",
    title="各地の最高気温と最低気温" 
)
fig.show()

In [ ]:
# データフレームをcsvファイルにして保存する
# 保存先ディレクトリ
save_dir = Path("output/#4")
file_name = input_file_name + ".csv"

save_path = save_dir / file_name
# 親ディレクトリが存在しないときは作成
# parents=True: 親ディレクトリも必要に応じて作成
# exist_ok=True: 既に存在していてもエラーにしない
save_path.parent.mkdir(parents=True, exist_ok=True)

target_df.to_csv(save_path, index=False)
print(f"データを {save_path} に保存しました。")